# Access DB2 on Cloud using Python

In [ ]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
!pip install ipython-sql

In [ ]:
import ibm_db

In [ ]:
dsn_hostname = "6667d8e9-9d4d-4ccb-ba32-21da3bb5aafc.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "fvk03629"        # e.g. "abc12345"
dsn_pwd = "kmkFF8KOKwG19HWj"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "30376"                # e.g. "32733" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

In [ ]:
# Ibm_db API uses the IBM Data Server Driver for ODBC and CLI APIs to connect to IBM DB2 and Informix.
# Create the database connection

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=6667d8e9-9d4d-4ccb-ba32-21da3bb5aafc.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud;PORT=30376;PROTOCOL=TCPIP;UID=fvk03629;PWD=kmkFF8KOKwG19HWj;SECURITY=SSL;


In [ ]:
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  fvk03629 on host:  6667d8e9-9d4d-4ccb-ba32-21da3bb5aafc.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud


Unable to connect:  [IBM][CLI Driver] SQL1336N  

The remote host "6667d8e9-9d4d-4ccb-ba32-21da3bb5aafc.c1ogj3sd0tgtu0lqde00.databases.ap" was not found.  

SQLSTATE=08001 SQLCODE=-1336


ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(10), LNAME VARCHAR(10), CITY VARCHAR(20), CCODE CHAR(2)

In [ ]:
dropQuery = "drop table INSTRUCTOR"

try:
  dropStmt = ibm_db.exec_immediate(conn, dropQuery)
except Exception as e:
  print(e)

[IBM][CLI Driver][DB2/LINUXX8664] SQL0204N  "FVK03629.INSTRUCTOR" is an undefined name.  SQLSTATE=42704 SQLCODE=-204


In [ ]:
createQuery = "create table INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(10), LNAME VARCHAR(10), CITY VARCHAR(20), CCODE CHAR(2))"

createStmt = ibm_db.exec_immediate(conn, createQuery)
#REPLACED replace_with_name_of_execution_method WITH exec_immediate

In [ ]:
# single insert
insertQuery = """INSERT INTO INSTRUCTOR VALUES (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"""
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [ ]:
# multiple insert
insertQuery2 = """INSERT INTO INSTRUCTOR VALUES 
(2, 'Raul', 'Chong', 'Markham', 'CA'),
(3, 'Hima', 'Vasudevan', 'Chicago', 'US')"""
insertStmt = ibm_db.exec_immediate(conn, insertQuery2)

In [ ]:
selectQuery = "select * from INSTRUCTOR"
selectStmt = ibm_db.exec_immediate(conn, selectQuery)
ibm_db.fetch_both(selectStmt)

{0: 1,
 1: 'Rav',
 2: 'Ahuja',
 3: 'TORONTO',
 4: 'CA',
 'CCODE': 'CA',
 'CITY': 'TORONTO',
 'FNAME': 'Rav',
 'ID': 1,
 'LNAME': 'Ahuja'}

In [ ]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (f'ID:{ibm_db.result(selectStmt, 0)} FNAME:{ibm_db.result(selectStmt, "FNAME")}')

ID:2 FNAME:Raul
ID:3 FNAME:Hima


In [ ]:
updateQuery = "update INSTRUCTOR set CITY='MOOSETOWN' where FNAME='Rav'"
updateStmt = ibm_db.exec_immediate(conn, updateQuery)

Retrieve data into Pandas


In [ ]:
import pandas as pd
import ibm_db_dbi as idd

In [ ]:
pconn = idd.Connection(conn)

In [ ]:
selectQuery = "select * from INSTRUCTOR"
df = pd.read_sql(selectQuery, pconn)

In [ ]:
df

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,TORONTO,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [ ]:
ibm_db.close(conn)

True